In [46]:
import cv2
import numpy as np
from scipy.spatial import distance
import cPickle as pickle
from skimage.feature import local_binary_pattern
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize

In [4]:
def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

bin_n = 16

def hog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist

In [5]:
def all_files(list_file):
    file_names = []
    with open(list_file, 'rb') as f:
        for line in f:
            file_names.append(line.split()[0])
    return file_names

In [55]:
train_files = all_files('/Users/ecsark/Documents/visualdb/project/wikiart/train.txt')
test_files = all_files('/Users/ecsark/Documents/visualdb/project/wikiart/test.txt')

In [11]:
hog_train = [hog(cv2.imread(img)) for img in train_files]

In [56]:
hog_test = [hog(cv2.imread(img)) for img in test_files]

In [17]:
pickle.dump(hog_train, 
            open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/hog_train.pk', 'wb'), 
            protocol=-1)

In [57]:
pickle.dump(hog_test, 
            open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/hog_test.pk', 'wb'), 
            protocol=-1)

In [8]:
def findMostSimilar(X, x_query, k=5):
    dist = distance.cdist(X, np.array([x_query]), 'euclidean')
    return dist.flatten().argsort()[:k]

In [19]:
[train_files[i] for i in findMostSimilar(all_features, hog(cv2.imread(train_files[7])), 10)]

['/Users/ecsark/Documents/visualdb/project/wikiart/images/theodore-rousseau_last-houses-of-port-en-bessin-calvados.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/salvador-dali_nude-on-the-plain-of-rosas.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/raphael-kirchner_boys-and-girls-at-sea-8.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/akseli-gallen-kallela_old-woman-with-a-cat-1885.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/octavio-ocampo_absents-of-the-mermaid.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/anna-ostroumova-lebedeva_leningrad-view-from-trinity-bridge-1926.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/jacek-yerka_road.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/childe-hassam_cotysville.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/volodymyr-orlovsky_summer-landscape-with-road.jpg',
 '/Users/ecsark/Documents/visualdb/project/wik

In [33]:
img = cv2.imread(train_files[8], cv2.CV_LOAD_IMAGE_GRAYSCALE)

In [34]:
h, w = img.shape[:2]
vis0 = np.zeros((h,w), np.float32)
vis0[:h, :w] = img
vis1 = cv2.dct(vis0)

In [35]:
vis1

array([[  1.26877758e+05,  -1.52201289e+04,   1.57428496e+04, ...,
          1.15846767e+01,  -4.10671377e+00,   3.18151116e+00],
       [ -9.53188184e+03,   1.38818555e+03,   6.98631396e+03, ...,
          1.03888807e+01,  -2.90842462e+00,  -1.67760658e+00],
       [ -9.18642480e+03,  -4.24782166e+02,   2.01197412e+03, ...,
          6.16781139e+00,  -4.81275940e+00,   3.98108172e+00],
       ..., 
       [  4.33764458e+00,  -1.10785561e+01,  -2.39274406e+00, ...,
          8.52350175e-01,  -9.14332032e-01,  -7.45342195e-01],
       [  7.33253288e+00,   8.94276047e+00,  -2.12687016e-01, ...,
         -3.96727212e-03,  -2.04966404e-02,  -7.63854757e-02],
       [ -6.65722847e-01,  -1.99235177e+00,   1.50952530e+00, ...,
         -1.01951337e+00,   1.14290619e+00,   1.29224643e-01]], dtype=float32)

In [43]:
extractor = cv2.DescriptorExtractor_create("OpponentSURF")

In [39]:
img = cv2.imread(train_files[7])
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT()
kp1, des1 = sift.detectAndCompute(img,None)

In [42]:
des1.shape

(2779, 128)

In [50]:
radius = 3
no_points = 8 * radius

def extract_lbp(fn):
    im_gray = cv2.cvtColor(cv2.imread(fn), cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(im_gray, no_points, radius, method='uniform')
    x = itemfreq(lbp.ravel())
    return x[:, 1]/sum(x[:, 1])

In [51]:
lbp_train = [extract_lbp(img) for img in train_files]

In [58]:
lbp_test = [extract_lbp(img) for img in test_files]

In [53]:
pickle.dump(lbp_train, 
            open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/lbp_train.pk', 'wb'), 
            protocol=-1)

In [60]:
pickle.dump(lbp_test, 
            open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/lbp_test.pk', 'wb'), 
            protocol=-1)

In [71]:
[train_files[i] for i in findMostSimilar(lbp_features, lbp_features[7], 10)]

['/Users/ecsark/Documents/visualdb/project/wikiart/images/theodore-rousseau_last-houses-of-port-en-bessin-calvados.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/nicholas-roerich_himalayas-63.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/william-adolphe-bouguereau_alone-in-the-world.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/albert-marquet_view-of-agay-1905.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/theodore-gericault_three-rumps-of-horses.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/eileen-agar_psychodelic-dream.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/zdislav-beksinski_untitled-668.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/ivan-vladimirov_flight-of-the-bourgeoisie-from-novorossiysk-in-1920.jpg',
 '/Users/ecsark/Documents/visualdb/project/wikiart/images/victor-borisov-musatov_silence-indoors-1900.jpg',
 '/Users/ecsark/Documents/visualdb/project/w